## import necessary packages 

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## read in CSVs 

In [2]:
calls_derecho = pd.read_csv('./data/911_Phone_Calls_Derecho_050320_050420.csv')
calls_tornado = pd.read_csv('./data/911_Phone_Calls_Tornado_030320.csv')
dispatch_derecho = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)
dispatch_tornado = pd.read_csv('./data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
zipcodes = gpd.read_file('./data/zipcodes.geojson')

## Rename columns 

In [3]:
calls_derecho = calls_derecho.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [4]:
calls_tornado = calls_tornado.rename(columns = {'Seizure DateTime': 'DateTime', 'CallTypeId': 'Call Type', 'ALI Latitude': 'Latitude', 'ALI Longitude': 'Longitude', 'Cell Tower Address': 'Cell Tower'})

In [5]:
dispatch_derecho = dispatch_derecho.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

In [6]:
dispatch_tornado = dispatch_tornado.rename(columns = {'IncidentDate': 'DateTime', 'Latitude1': 'Latitude', 'Longitude1': 'Longitude', 'IncidentTypeDescription1': 'Incident Description'})

## Add geometry points to data sets 

In [7]:
calls_derecho['Geometry'] = calls_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [8]:
calls_tornado['Geometry'] = calls_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [9]:
dispatch_derecho['Geometry'] = dispatch_derecho.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

In [10]:
dispatch_tornado['Geometry'] = dispatch_tornado.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)

## check data

In [11]:
calls_derecho.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802)
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062099999999 36.14310800000001)
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N,POINT (-87.05632900000001 36.100516)
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW,POINT (-86.927819 36.130021)
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE,POINT (-86.88188199999999 36.14238)


In [12]:
calls_tornado.head()

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708)
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.841904 36.159634)
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector,POINT (-86.708862 36.264153)
3,3/3/2020 0:36,911 Calls,NaN,NaN,526 FOUNDRY DR,POINT (nan nan)
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N,POINT (-86.72107700000001 36.233467)


In [13]:
dispatch_derecho.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715)
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081)
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD,POINT (-86.71687484 36.24802948)
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN,POINT (-86.71481656 36.25649117)
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD,POINT (-86.87983392 36.12968971)


In [14]:
dispatch_tornado.head()

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191)
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666)
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL,POINT (-86.8837274 36.17929946)
3,3/3/2020 12:39:40 AM,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.6899214 36.1511077)
4,3/3/2020 12:39:42 AM,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.84593411 36.17792604)


In [15]:
zipcodes.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37115,1,MADISON,178783.02488886821,596553400.57885742,"MULTIPOLYGON (((-86.68725 36.31821, -86.68722 ..."
1,37216,3,NASHVILLE,75820.997821400058,188884682.28344727,"MULTIPOLYGON (((-86.73451 36.23774, -86.73425 ..."
2,37204,9,NASHVILLE,93180.292250425613,200664795.51708984,"MULTIPOLYGON (((-86.77914 36.13424, -86.77923 ..."
3,37027,11,BRENTWOOD,159760.6942933173,174978422.04101562,"MULTIPOLYGON (((-86.81258 36.06319, -86.81263 ..."
4,37064,18,FRANKLIN,28995.828320601937,46969608.005737305,"MULTIPOLYGON (((-87.02197 36.01200, -87.02140 ..."


## shape check

In [16]:
calls_derecho.shape

(1549, 6)

In [17]:
calls_tornado.shape

(475, 6)

In [18]:
dispatch_derecho.shape

(627, 6)

In [19]:
dispatch_tornado.shape

(260, 6)

## count Cell Towers and Incident Descriptions

In [ ]:
calls_derecho['Cell Tower'].value_counts()

In [ ]:
calls_tornado['Cell Tower'].value_counts()

In [ ]:
dispatch_derecho['Incident Description'].value_counts()

In [ ]:
dispatch_tornado['Incident Description'].value_counts()

## count null values

In [ ]:
calls_derecho.isnull().sum()

In [ ]:
calls_tornado.isnull().sum()

In [ ]:
dispatch_derecho.isnull().sum()

In [ ]:
dispatch_tornado.isnull().sum()

map1 = sns.FacetGrid(dispactch_derecho, col = 'Incident Description')
map1.map(sns.barplot, "DateTime")

plt.hist(dispatch_derecho['Incident Description'], bins = 10);

## clean zipcodes data and create geodata frames for all datasets

In [20]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [21]:
derecho_calls_geo = gpd.GeoDataFrame(calls_derecho, 
                           crs = zipcodes.crs, 
                           geometry = calls_derecho['Geometry'])
type(derecho_calls_geo)

geopandas.geodataframe.GeoDataFrame

In [22]:
tornado_calls_geo = gpd.GeoDataFrame(calls_tornado, 
                           crs = zipcodes.crs, 
                           geometry = calls_tornado['Geometry'])
type(tornado_calls_geo)

geopandas.geodataframe.GeoDataFrame

In [23]:
dispatch_derecho_geo = gpd.GeoDataFrame(dispatch_derecho, 
                           crs = zipcodes.crs, 
                           geometry = dispatch_derecho['Geometry'])
type(dispatch_derecho_geo)

geopandas.geodataframe.GeoDataFrame

In [24]:
dispatch_tornado_geo = gpd.GeoDataFrame(dispatch_tornado, 
                           crs = zipcodes.crs, 
                           geometry = dispatch_tornado['Geometry'])
type(dispatch_tornado_geo)

geopandas.geodataframe.GeoDataFrame

## merge data sets with zipcodes

In [25]:
derecho_calls_zip = gpd.sjoin(derecho_calls_geo, zipcodes, op = 'within')
derecho_calls_zip

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry,geometry,index_right,zip,po_name
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802),POINT (-86.71685 36.24780),0,37115,MADISON
68,5/3/2020 16:44,911 Calls,36.265547,-86.736116,524 BOYDS HILLTOP DR - SW,POINT (-86.73611600000001 36.265547),POINT (-86.73612 36.26555),0,37115,MADISON
78,5/3/2020 16:46,911 Calls,36.250710,-86.689766,607E LARKIN SPRINGS RD - SW,POINT (-86.68976600000001 36.25071),POINT (-86.68977 36.25071),0,37115,MADISON
79,5/3/2020 16:46,911 Calls,36.259485,-86.678717,96 D VANDIVER DR - SECTOR S,POINT (-86.67871700000001 36.259485),POINT (-86.67872 36.25948),0,37115,MADISON
100,5/3/2020 16:51,911 Calls,36.253467,-86.730366,619 DUE WEST AVE. W - NE SECTOR,POINT (-86.730366 36.253467),POINT (-86.73037 36.25347),0,37115,MADISON
...,...,...,...,...,...,...,...,...,...,...
829,5/3/2020 19:11,911 Calls,36.165276,-86.780491,315 4TH AV N,POINT (-86.780491 36.165276),POINT (-86.78049 36.16528),24,37219,NASHVILLE
880,5/3/2020 19:20,911 Calls,36.168324,-86.783089,500 5TH AV - SECTOR SW N,POINT (-86.783089 36.168324),POINT (-86.78309 36.16832),24,37219,NASHVILLE
892,5/3/2020 19:22,911 Calls,36.165715,-86.782079,315-B 4TH AV - SECTOR NW N,POINT (-86.78207900000001 36.16571500000001),POINT (-86.78208 36.16572),24,37219,NASHVILLE
1504,5/3/2020 23:25,911 Calls,36.004472,-86.695497,1598-A RAGSDALE RD - SECTOR N,POINT (-86.695497 36.004472),POINT (-86.69550 36.00447),14,37027,BRENTWOOD


In [26]:
tornado_calls_zip = gpd.sjoin(tornado_calls_geo, zipcodes, op = 'within')
tornado_calls_zip

,DateTime,Call Type,Latitude,Longitude,Cell Tower,Geometry,geometry,index_right,zip,po_name
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708),POINT (-86.66578 36.07271),51,37013,ANTIOCH
10,3/3/2020 0:38,911 Calls,36.052055,-86.648331,745 BELL RD - OMNI Sector,POINT (-86.648331 36.052055),POINT (-86.64833 36.05205),51,37013,ANTIOCH
64,3/3/2020 0:53,911 Calls,36.050048,-86.650629,5646 AMALIE DR - SE,POINT (-86.65062900000001 36.050048),POINT (-86.65063 36.05005),51,37013,ANTIOCH
194,3/3/2020 1:29,911 Calls,36.068298,-86.681550,3930 APACHE TRAIL - SE SECTOR,POINT (-86.68155 36.068298),POINT (-86.68155 36.06830),51,37013,ANTIOCH
234,3/3/2020 1:40,911 Calls,36.058578,-86.699745,5067 COLEMONT DR - N,POINT (-86.69974499999999 36.058578),POINT (-86.69974 36.05858),51,37013,ANTIOCH
...,...,...,...,...,...,...,...,...,...,...
444,3/3/2020 4:25,911 Calls,36.190487,-86.833237,2111 Summitt Ave - Sector SW,POINT (-86.833237 36.190487),POINT (-86.83324 36.19049),27,37218,NASHVILLE
209,3/3/2020 1:34,911 Calls,36.308312,-86.720581,1000 S CARTWRIGHT ST - SECTOR SW,POINT (-86.72058100000001 36.308312),POINT (-86.72058 36.30831),8,37072,GOODLETTSVILLE
323,3/3/2020 2:10,911 Calls,36.311104,-86.707214,456 MOSS TRL - SE Sector,POINT (-86.70721400000001 36.311104),POINT (-86.70721 36.31110),8,37072,GOODLETTSVILLE
328,3/3/2020 2:12,911 Calls,36.311102,-86.707213,WIRELESS CALLER,POINT (-86.707213 36.31110200000001),POINT (-86.70721 36.31110),8,37072,GOODLETTSVILLE


In [27]:
dispatch_derecho_zip = gpd.sjoin(dispatch_derecho_geo, zipcodes, op = 'within')
dispatch_derecho_zip

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry,geometry,index_right,zip,po_name
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715),POINT (-86.96253 36.04688),7,37221,NASHVILLE
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081),POINT (-86.94068 36.08249),7,37221,NASHVILLE
17,5/3/2020 4:39:57 PM,8715 OLD HARDING PKE,36.012524,-87.024584,ELECTRICAL HAZARD,POINT (-87.02458403999999 36.01252365),POINT (-87.02458 36.01252),7,37221,NASHVILLE
88,5/3/2020 5:12:40 PM,7205 MARK DR,36.065849,-86.928772,ELECTRICAL HAZARD,POINT (-86.92877244 36.06584867),POINT (-86.92877 36.06585),7,37221,NASHVILLE
116,5/3/2020 5:25:03 PM,1651 CHICKERING RD,36.059848,-86.874532,ENTRAPMENT / TRAPPED,POINT (-86.87453247000001 36.05984766),POINT (-86.87453 36.05985),7,37221,NASHVILLE
...,...,...,...,...,...,...,...,...,...,...
409,5/3/2020 8:00:32 PM,3531 HUNTLAND DR,36.274720,-86.842008,DIFFICULTY SPEAKING BETWEEN BREATHS,POINT (-86.84200841000001 36.27471998),POINT (-86.84201 36.27472),18,37189,WHITES CREEK
460,5/3/2020 8:40:46 PM,1211 MEDICAL CENTER DR,36.142407,-86.801598,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.80159753 36.14240652),POINT (-86.80160 36.14241),30,37232,NASHVILLE
604,5/3/2020 11:30:49 PM,2ND AV N / JUNIOR GILLIAM WAY,36.174401,-86.783024,CHEST PAIN - CLAMMY OR COLD SWEATS,POINT (-86.78302399 36.17440126),POINT (-86.78302 36.17440),36,37201,NASHVILLE
606,5/3/2020 11:39:35 PM,2022 ROSA L PARKS BLVD,36.188362,-86.798090,TRAUMATIC INJURIES NOT DANGEROUS BODY AREA,POINT (-86.79808982 36.18836237),POINT (-86.79809 36.18836),45,37228,NASHVILLE


In [28]:
dispatch_tornado_zip = gpd.sjoin(dispatch_tornado_geo, zipcodes, op = 'within')
dispatch_tornado_zip

,DateTime,Location,Latitude,Longitude,Incident Description,Geometry,geometry,index_right,zip,po_name
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191),POINT (-86.89405 36.18200),12,37209,NASHVILLE
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666),POINT (-86.89498 36.17567),12,37209,NASHVILLE
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL,POINT (-86.8837274 36.17929946),POINT (-86.88373 36.17930),12,37209,NASHVILLE
17,3/3/2020 12:44:49 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,BUILDING ENTRAPMENT UNCONFIRMED,POINT (-86.89497756999999 36.175666),POINT (-86.89498 36.17567),12,37209,NASHVILLE
19,3/3/2020 12:45:10 AM,7135 CENTENNIAL PL,36.176893,-86.879949,INSIDE FUEL ODOR,POINT (-86.87994904999999 36.17689288),POINT (-86.87995 36.17689),12,37209,NASHVILLE
...,...,...,...,...,...,...,...,...,...,...
231,3/3/2020 4:06:14 AM,200 E WEBSTER ST,36.251077,-86.716657,CITIZEN ASSIST NON MEDICAL,POINT (-86.71665675 36.25107722),POINT (-86.71666 36.25108),0,37115,MADISON
234,3/3/2020 4:16:53 AM,368 BRITISH WOODS DR,36.092959,-86.654322,DIFFICULTY SPEAKING BETWEEN BREATHS,POINT (-86.65432195 36.0929593),POINT (-86.65432 36.09296),17,37217,NASHVILLE
236,3/3/2020 4:21:34 AM,115 LAURA AV,36.234591,-86.630412,DIFFICULTY SPEAKING BETWEEN BREATHS OTHER LUNG...,POINT (-86.63041192999999 36.23459081),POINT (-86.63041 36.23459),22,37138,OLD HICKORY
240,3/3/2020 4:27:57 AM,200 ATHENS WAY,36.195772,-86.799250,SMOKE DETECTOR ALARM COMMERCIAL INDUSTRIAL B...,POINT (-86.79925022 36.19577205),POINT (-86.79925 36.19577),45,37228,NASHVILLE


## nashville map 

In [30]:
nashville = (36.16784, -86.77816)

nash_map = folium.Map(location = nashville, zoom_start = 10)

In [31]:
nash_map

In [32]:
poly_zip=zipcodes[zipcodes['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
poly_zip.shape

(11, 3)

In [33]:
tcalls_in_nw = tornado_calls_zip[tornado_calls_zip['zip'].isin(['37207', '37072', '37189', '37218','37015','37080'])]
tcalls_in_nw.shape

(43, 10)

In [34]:
cluster_map = MarkerCluster().add_to(cluster_map)

folium.GeoJson(poly_zip).add_to(cluster_map)

for row_index, row_values in tcalls_in_nw.iterrows():
    loc = [row_values['lat'], row_values['lng']]
    pop = str(row_values['address'])
    icon=folium.Icon(color="red",icon="phone", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(cluster_map)

    
#cluster_map_nw.save('../maps/clusternw.html')

#cluster_map_nw


NameError: name 'cluster_map' is not defined

folium.GeoJson(districts['geometry']).add_to(nash_map)
folium.TileLayer('cartodbpositron').add_to(nash_map)
marker_cluster = MarkerCluster().add_to(nash_map)
for row_index, row_values in districts_development.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values[['Project Name', 'council_district']])
    icon=folium.Icon(color="darkpurple",icon="map-pin", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(marker_cluster)

In [ ]:
#poly = zipcodes.loc[zipcodes['zip'] == '37207']
#polygon37207.shape